In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, optimizers, callbacks
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D

import numpy as np
import matplotlib.pyplot as plt



import matplotlib.pyplot as plt
import numpy as np
import os



In [5]:
# 1. 데이터 불러오기

In [ ]:
# 2. 모델 구조 설정
model = Sequential()
model.add(Dense(n, input_dim=n, activation='relu'))
model.add(Dense(n, activation='softmax'))

model.summary()

In [ ]:
# 3. 모델 실행 환경 설정
model.compile(loss = "binary_crossentropy", optimizer="adam", metrics=['accuracy'])


In [ ]:
# 4. 모델 최적화
model_path = "./model/project_model.keras"
checkpointer = ModelCheckpoint(filepath = model_path, monitor = "val_loss", verbose = 1, save_best_only = True)
earlystopping_callback = EarlyStopping(monitor = "val_loss", patience = 10)

In [ ]:
# 5. 모델 실행
history = model.fit(X_train, y_train, validate_stplit = 0.2, epoch = 30, batch_size = 200, verbose = 1, callbacks = [earlystopping_callback, checkpointer])
print("\n Test Acc: %.4f" % (model.evaluate(X_test, y_test)[1]))

In [3]:
# 6. 시각화
# 6-1. val_loss, loss 저장

y_vloss = history.history['val_loss']
y_loss = history.history['loss']

# 6-2. plt로 그래프 표현
x_len = np.arange(len(y_loss)) #y_loss의 정수 배열 생성, 즉, epoch을 의미
plt.plot(x_len, y_vloss, marker = ".", c="red", label="Validationset_loss")
plt.plot(x_len, y_loss, marker = ".", c="blue", label="Trainset_loss")
plt.legend(los = 'uppter right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
# 7. 전이학습

# 7-1. 모델 불러오기
transfer_model = models.load_model(model_path)
transfer_model.trainable = False

# 7-2. fine-tuning 설정
finetune_model = models.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation = 'relu'))

# 7-3. 모델 실행 환경 설정
finetune_model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ['auroc'])

# 7-4. 모델 최적화
ft_model_path = "./model/project_model_finetuned.keras"
ft_checkpointer = ModelCheckpoint(filepath = ft_model_path, monitor = "val_loss", verbose = 1, save_best_only = True)
ft_earlystopping_callback = EarlyStopping(monitor = "val_loss", patience = 10)

# 7-5. 모델 실행
history = finetune_model.fit(X_train, y_train, epochs = 30, callbacks = [ft_earlystopping_callback, ft_checkpointer])

#7-6. 정확도 확인
print("\n Test Accuracy: %.4f" % (finetune_model.evaluate(X_test, y_test)[1]))